## Done

## ToDo
 - [ ] Create County-FacilityName => CCN/CMS Number
 - [ ] Manually Clean up some of the CMS Numbers
 - [ ] Create functions for some of the cells

## Done

In [1]:
import pandas as pd
import numpy as np
import json
import requests
import lxml.html as lh
import re
from bs4 import BeautifulSoup

In [2]:
url = 'https://www.michigan.gov/coronavirus/0,9753,7-406-98163_98173-526911--,00.html'
url = 'https://www.michigan.gov/coronavirus/0,9753,7-406-98163_98173-526911--,00.html'

# Wayback Machine work!!!
#url = 'https://web.archive.org/web/20201130212042/https://www.michigan.gov/coronavirus/0,9753,7-406-98163_98173-526911--,00.html'
#url = 'https://web.archive.org/web/20200720171507/https://www.michigan.gov/coronavirus/0,9753,7-406-98163_98173-526911--,00.html'
# TODO - Wayback machine adds two extra tables at top we need to skip past
page = requests.get(url)
soup = BeautifulSoup(page.text, 'lxml')

In [3]:
tables = soup.find_all('table')

tmp = re.split('/| ', tables[0].find_all('caption')[0].get_text())
reporting_date = '-'.join(tmp[0:3])
ctp_date_format = tmp[2]+tmp[0]+tmp[1]
print(reporting_date)

02-08-2021


In [4]:
# Create list of facilities
rows = tables[2].find_all('tr')
# Get List of fields from Header Row
fieldList = []
fields = rows[0].find_all('th')
for field in fields:
    fieldList.append(field.get_text())
#assert fieldList == ['FACILITY NAME', 'COUNTY','TYPE', 'NEW RESIDENT CASES', 'TOTAL RESIDENT CASES', 'NEW RESIDENT DEATHS', 'TOTAL RESIDENT DEATHS',
#                     'NEW STAFF CASES', 'TOTAL STAFF CASES', 'NEW STAFF DEATHS', 'TOTAL STAFF DEATHS']
num_facilities = 0
facilities = []
first_row = True
for row in rows:
    if first_row:
        first_row = False
        continue
    facility = {}
    tds = row.find_all('td')
    num_reported_vals = 0
    facility['ReportingDate'] = ctp_date_format
    for i in range(len(tds)):
        val = tds[i].get_text()
        if val == '.':
            val = float('Nan')
        else:
            num_reported_vals = num_reported_vals +1
        facility[fieldList[i]] = val   
    facilities.append(facility)
print(len(facilities))

1350


In [5]:
# facilities_df = pd.read_json(json.dumps(facilities), dtype={'ReportingDate': np.str, 'FACILITY NAME': str, 'COUNTY': str, 'TYPE': str,
#                                       'NEW RESIDENT CASES': float, 'TOTAL RESIDENT CASES': float, 'NEW RESIDENT DEATHS': float,
#                       'TOTAL RESIDENT DEATHS': float, 'NEW STAFF CASES': float, 'TOTAL STAFF CASES': float,
#                       'NEW STAFF DEATHS': float, 'TOTAL STAFF DEATHS': float})
facilities_df = pd.read_json(json.dumps(facilities), dtype=True)

In [6]:
def MI_2_CTP_Facility_Type_Mapping(type):
    if type == 'SNF':
        return 'Nursing Home'
    else:
        return 'Other'

In [7]:
facilities_df.columns

Index(['ReportingDate', 'FACILITY NAME', 'COUNTY', 'TYPE', 'COMPLIANCE',
       'NEW RESIDENT CASES', 'TOTAL RESIDENT CASES', 'NEW RESIDENT DEATHS',
       'TOTAL RESIDENT DEATHS', 'NEW STAFF CASES', 'TOTAL STAFF CASES',
       'NEW STAFF DEATHS', 'TOTAL STAFF DEATHS'],
      dtype='object')

In [8]:
#facilities_df.sort_values(by='TYPE', ascending=False).head(30)
MI_empty_cols = ['City', 'State/Fed Regulated', 'State Facility ID', 'CMS Facility ID', 'Date outbreak opened', 'Date outreak closed', 'Outbreak Status','Resident Census',
                 'TOTAL Resident Probable Positives', 'TOTAL Resident Probable Deaths', 'TOTAL Staff Probable Positives', 'TOTAL Staff Probable Deaths',
                 'TOTAL Resident/Staff Positives', 'TOTAL Resident/Staff Probable Positives', 'TOTAL Resident/Staff Deaths', 'TOTAL Resident/Staff Probable Deaths',
                 'NEW Resident Probable Positives', 'NEW Resident Probable Deaths', 'NEW Staff Probable Positives', 'NEW Staff Probable Deaths',
                 'NEW Resident/Staff Positives', 'NEW Resident/Staff Probable Positives', 'NEW Resident/Staff Deaths', 'NEW Resident/Staff Probable Deaths',
                   'Residents Tested', 'Staff Tested', 'Personal Protective Equipment']

for col in MI_empty_cols:
    facilities_df[col] = ''
facilities_df['State'] = 'MI'
facilities_df['CTP Facility Categorization'] = facilities_df['TYPE'].apply(MI_2_CTP_Facility_Type_Mapping)
facilities_df.head()

,ReportingDate,FACILITY NAME,COUNTY,TYPE,COMPLIANCE,NEW RESIDENT CASES,TOTAL RESIDENT CASES,NEW RESIDENT DEATHS,TOTAL RESIDENT DEATHS,NEW STAFF CASES,...,NEW Staff Probable Deaths,NEW Resident/Staff Positives,NEW Resident/Staff Probable Positives,NEW Resident/Staff Deaths,NEW Resident/Staff Probable Deaths,Residents Tested,Staff Tested,Personal Protective Equipment,State,CTP Facility Categorization
0,20210208,Jamieson Nursing Home,ALCONA,SNF,,0.0,19.0,0.0,4.0,0.0,...,,,,,,,,,MI,Nursing Home
1,20210208,Lincoln Haven Nursing & Rehab. Community,ALCONA,SNF,,0.0,28.0,0.0,7.0,0.0,...,,,,,,,,,MI,Nursing Home
2,20210208,Medilodge of Munising,ALGER,SNF,,0.0,0.0,0.0,0.0,0.0,...,,,,,,,,,MI,Nursing Home
3,20210208,Allegan County Medical Care Community,ALLEGAN,SNF,,0.0,1.0,0.0,0.0,0.0,...,,,,,,,,,MI,Nursing Home
4,20210208,Beacon Home at The Oaks,ALLEGAN,AFC,,0.0,0.0,0.0,0.0,0.0,...,,,,,,,,,MI,Other


In [9]:
CTP_cols = ['ReportingDate', 'State', 'COUNTY', 
            'City',
            'FACILITY NAME', 'TYPE',
            'CTP Facility Categorization',
            'State/Fed Regulated',
            'State Facility ID',
            'CMS Facility ID',
            'Date outbreak opened', 'Date outreak closed', 'Outbreak Status','Resident Census',
            
'TOTAL RESIDENT CASES', # 'Resident Positives ',
          'TOTAL Resident Probable Positives',
'TOTAL RESIDENT DEATHS', # 'Resident Deaths',
         'TOTAL Resident Probable Deaths',
'TOTAL STAFF CASES', #'Staff Positive',
         'TOTAL Staff Probable Positives',
'TOTAL STAFF DEATHS', #'TOTAL Staff Deaths',
         'TOTAL Staff Probable Deaths',
 'TOTAL Resident/Staff Positives',
 'TOTAL Resident/Staff Probable Positives',
 'TOTAL Resident/Staff Deaths',
 'TOTAL Resident/Staff Probable Deaths',
 
'NEW RESIDENT CASES', #            'Resident Positives',
     'NEW Resident Probable Positives',
 'NEW RESIDENT DEATHS',
     'NEW Resident Probable Deaths',
 'NEW STAFF CASES',
     'NEW Staff Probable Positives',
 'NEW STAFF DEATHS',
     'NEW Staff Probable Deaths',
 'NEW Resident/Staff Positives',
 'NEW Resident/Staff Probable Positives',
 'NEW Resident/Staff Deaths',
 'NEW Resident/Staff Probable Deaths',
 'Residents Tested',
 'Staff Tested',
 'Personal Protective Equipment',
           'COMPLIANCE']

In [10]:
facilities_df.columns

Index(['ReportingDate', 'FACILITY NAME', 'COUNTY', 'TYPE', 'COMPLIANCE',
       'NEW RESIDENT CASES', 'TOTAL RESIDENT CASES', 'NEW RESIDENT DEATHS',
       'TOTAL RESIDENT DEATHS', 'NEW STAFF CASES', 'TOTAL STAFF CASES',
       'NEW STAFF DEATHS', 'TOTAL STAFF DEATHS', 'City', 'State/Fed Regulated',
       'State Facility ID', 'CMS Facility ID', 'Date outbreak opened',
       'Date outreak closed', 'Outbreak Status', 'Resident Census',
       'TOTAL Resident Probable Positives', 'TOTAL Resident Probable Deaths',
       'TOTAL Staff Probable Positives', 'TOTAL Staff Probable Deaths',
       'TOTAL Resident/Staff Positives',
       'TOTAL Resident/Staff Probable Positives',
       'TOTAL Resident/Staff Deaths', 'TOTAL Resident/Staff Probable Deaths',
       'NEW Resident Probable Positives', 'NEW Resident Probable Deaths',
       'NEW Staff Probable Positives', 'NEW Staff Probable Deaths',
       'NEW Resident/Staff Positives', 'NEW Resident/Staff Probable Positives',
       'NEW Residen

In [11]:
facilities_df[CTP_cols].to_csv('Reporting_data/MI_' + reporting_date + 'LTC_data.csv', index=False)

In [12]:
facilities_df.dtypes

ReportingDate                                int64
FACILITY NAME                               object
COUNTY                                      object
TYPE                                        object
COMPLIANCE                                  object
NEW RESIDENT CASES                         float64
TOTAL RESIDENT CASES                       float64
NEW RESIDENT DEATHS                        float64
TOTAL RESIDENT DEATHS                      float64
NEW STAFF CASES                            float64
TOTAL STAFF CASES                          float64
NEW STAFF DEATHS                           float64
TOTAL STAFF DEATHS                         float64
City                                        object
State/Fed Regulated                         object
State Facility ID                           object
CMS Facility ID                             object
Date outbreak opened                        object
Date outreak closed                         object
Outbreak Status                

In [13]:
facilities_df

,ReportingDate,FACILITY NAME,COUNTY,TYPE,COMPLIANCE,NEW RESIDENT CASES,TOTAL RESIDENT CASES,NEW RESIDENT DEATHS,TOTAL RESIDENT DEATHS,NEW STAFF CASES,...,NEW Staff Probable Deaths,NEW Resident/Staff Positives,NEW Resident/Staff Probable Positives,NEW Resident/Staff Deaths,NEW Resident/Staff Probable Deaths,Residents Tested,Staff Tested,Personal Protective Equipment,State,CTP Facility Categorization
0,20210208,Jamieson Nursing Home,ALCONA,SNF,,0.0,19.0,0.0,4.0,0.0,...,,,,,,,,,MI,Nursing Home
1,20210208,Lincoln Haven Nursing & Rehab. Community,ALCONA,SNF,,0.0,28.0,0.0,7.0,0.0,...,,,,,,,,,MI,Nursing Home
2,20210208,Medilodge of Munising,ALGER,SNF,,0.0,0.0,0.0,0.0,0.0,...,,,,,,,,,MI,Nursing Home
3,20210208,Allegan County Medical Care Community,ALLEGAN,SNF,,0.0,1.0,0.0,0.0,0.0,...,,,,,,,,,MI,Nursing Home
4,20210208,Beacon Home at The Oaks,ALLEGAN,AFC,,0.0,0.0,0.0,0.0,0.0,...,,,,,,,,,MI,Other
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1345,20210208,Pleasant Lake Lodge North,WEXFORD,AFC,NOT IN COMPLIANCE,NaN,0.0,NaN,0.0,NaN,...,,,,,,,,,MI,Other
1346,20210208,Pleasant Lake Lodge South,WEXFORD,AFC,NOT IN COMPLIANCE,NaN,0.0,NaN,0.0,NaN,...,,,,,,,,,MI,Other
1347,20210208,Samaritas Senior Living Cadillac,WEXFORD,SNF,,0.0,25.0,0.0,2.0,0.0,...,,,,,,,,,MI,Nursing Home
1348,20210208,Serenity Adult Foster Care,WEXFORD,AFC,,0.0,0.0,0.0,0.0,0.0,...,,,,,,,,,MI,Other


In [14]:
cols = ['NEW RESIDENT CASES', 'TOTAL RESIDENT CASES', 'NEW RESIDENT DEATHS',
       'TOTAL RESIDENT DEATHS', 'NEW STAFF CASES', 'TOTAL STAFF CASES',
       'NEW STAFF DEATHS', 'TOTAL STAFF DEATHS']
print(reporting_date)
print(facilities_df['TYPE'].value_counts())
print(facilities_df[cols].sum().astype(int))
# print(facilities_df[cols].sum())

02-08-2021
AFC    601
SNF    440
HFA    309
Name: TYPE, dtype: int64
NEW RESIDENT CASES         154
TOTAL RESIDENT CASES     23766
NEW RESIDENT DEATHS         64
TOTAL RESIDENT DEATHS     5470
NEW STAFF CASES            219
TOTAL STAFF CASES        19407
NEW STAFF DEATHS             2
TOTAL STAFF DEATHS          76
dtype: int32


In [15]:
facilities_df[CTP_cols]

,ReportingDate,State,COUNTY,City,FACILITY NAME,TYPE,CTP Facility Categorization,State/Fed Regulated,State Facility ID,CMS Facility ID,...,NEW STAFF DEATHS,NEW Staff Probable Deaths,NEW Resident/Staff Positives,NEW Resident/Staff Probable Positives,NEW Resident/Staff Deaths,NEW Resident/Staff Probable Deaths,Residents Tested,Staff Tested,Personal Protective Equipment,COMPLIANCE
0,20210208,MI,ALCONA,,Jamieson Nursing Home,SNF,Nursing Home,,,,...,0.0,,,,,,,,,
1,20210208,MI,ALCONA,,Lincoln Haven Nursing & Rehab. Community,SNF,Nursing Home,,,,...,0.0,,,,,,,,,
2,20210208,MI,ALGER,,Medilodge of Munising,SNF,Nursing Home,,,,...,0.0,,,,,,,,,
3,20210208,MI,ALLEGAN,,Allegan County Medical Care Community,SNF,Nursing Home,,,,...,0.0,,,,,,,,,
4,20210208,MI,ALLEGAN,,Beacon Home at The Oaks,AFC,Other,,,,...,0.0,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1345,20210208,MI,WEXFORD,,Pleasant Lake Lodge North,AFC,Other,,,,...,NaN,,,,,,,,,NOT IN COMPLIANCE
1346,20210208,MI,WEXFORD,,Pleasant Lake Lodge South,AFC,Other,,,,...,NaN,,,,,,,,,NOT IN COMPLIANCE
1347,20210208,MI,WEXFORD,,Samaritas Senior Living Cadillac,SNF,Nursing Home,,,,...,0.0,,,,,,,,,
1348,20210208,MI,WEXFORD,,Serenity Adult Foster Care,AFC,Other,,,,...,0.0,,,,,,,,,


In [16]:
#CTP_cols = 'Date Collected	State 	County	City	Facility Name	State Facility Type	CTP Facility Categorization	State/Fed Regulated	State Facility ID	CMS Facility ID	Date outbreak opened	Date outreak closed	Outbreak Status	Resident Census	Resident Positives	Resident Probable Positives	Resident Deaths	Resident Probable Deaths	Staff Positive	Staff Probable Positives	Staff Deaths	Staff Probable Deaths	Resident/Staff Positives	Resident/Staff Probable Positives	Resident/Staff Deaths	Resident/Staff Probable Deaths	Resident Positives	Resident Probable Positives	Resident Deaths	Resident Probable Deaths	Staff Positive	Staff Probable Positives	Staff Deaths	Staff Probable Deaths	Resident/Staff Positives	Resident/Staff Probable Positives	Resident/Staff Deaths	Resident/Staff Probable Deaths	Residents Tested	Staff Tested	Personal Protective Equipment'


In [17]:
#CTP_cols.split('\t')